In [1]:
import numpy as np
import pandas as pd

from astropy import coordinates
import astropy.units as u
import astroquery
from astroquery.irsa import Irsa
from astroquery.vizier import Vizier

Vizier.ROW_LIMIT = -1

import matplotlib.pyplot as plt

%matplotlib inline

# Get the data

In [2]:
obj = ["3C 454.3", 343.49062, 16.14821, 4./60.]
# name, ra, dec, radius of cone
obj_name = obj[0]
obj_ra   = obj[1]
obj_dec  = obj[2]
cone_radius  = obj[3]

In [3]:
obj_coord = coordinates.SkyCoord(ra=obj_ra, dec=obj_dec, unit=(u.deg, u.deg), frame="icrs")

In [4]:
data_2mass = Irsa.query_region(obj_coord, catalog="fp_psc", radius=cone_radius * u.deg)
data_wise  = Irsa.query_region(obj_coord, catalog="allwise_p3as_psd", radius=cone_radius * u.deg)
data_galex = Vizier.query_region(obj_coord, catalog='II/335', radius=cone_radius * u.deg)
data_galex = data_galex[0]

In [9]:
num_2mass = len(data_2mass)
num_wise  = len(data_wise)
num_galex = len(data_galex)

print("Number of object in (2MASS, WISE, GALEX): ", num_2mass, num_wise, num_galex)

Number of object in (2MASS, WISE, GALE):  44 218 41


In [10]:
num_match_2mass_wise  = 0
num_match_2mass_galex = 0
num_match_wise_galex  = 0
sep_min = 2 # minimum separation in arcsec

## match: from 2MASS to WISE and GALEX
for i, idata in enumerate(data_2mass):
    ra_2mass  = idata['ra']
    dec_2mass = idata['dec']
    c_2mass = coordinates.SkyCoord(ra=ra_2mass, dec=dec_2mass, unit=(u.deg, u.deg), frame="icrs")
    
    #####
    # check in WISE cat
    match_wise = False
    for j, jdata in enumerate(data_wise):
        ra_wise  = jdata['ra']
        dec_wise = jdata['dec']
        c_wise = coordinates.SkyCoord(ra=ra_wise, dec=dec_wise, unit=(u.deg, u.deg), frame="icrs")
        
        sep_2mass_wise = c_2mass.separation(c_wise)
        
        if (sep_2mass_wise <= sep_min * u.arcsec):
            match_wise = True
            break
    
    if match_wise:
        num_match_2mass_wise  += 1
    
    #####
    # check in GALEX cat
    match_galex = False
    for k, kdata in enumerate(data_galex):
        ra_galex  = kdata['RAJ2000']
        dec_galex = kdata['DEJ2000']
        c_galex = coordinates.SkyCoord(ra=ra_galex, dec=dec_galex, unit=(u.deg, u.deg), frame="icrs")
        
        sep_2mass_galex = c_2mass.separation(c_galex)
        
        if (sep_2mass_galex < sep_min * u.arcsec):
            match_galex = True
            break
    
    if match_galex:
        num_match_2mass_galex += 1

#####
# check WISE in GALEX cat
for j, jdata in enumerate(data_wise):
    ra_wise  = jdata['ra']
    dec_wise = jdata['dec']
    c_wise = coordinates.SkyCoord(ra=ra_wise, dec=dec_wise, unit=(u.deg, u.deg), frame="icrs")
    
    match_wise_galex = False
    for k, kdata in enumerate(data_galex):
        ra_galex  = kdata['RAJ2000']
        dec_galex = kdata['DEJ2000']
        c_galex = coordinates.SkyCoord(ra=ra_galex, dec=dec_galex, unit=(u.deg, u.deg), frame="icrs")
    
        sep_wise_galex = c_wise.separation(c_galex)
        
        if (sep_wise_galex <= sep_min * u.arcsec):
            match_wise_galex = True
            break
    
    if match_wise_galex:
        num_match_wise_galex  += 1


print("Coordinate Match (2MASS-WISE, 2MASS-GALEX, WISE-GALEX): ", num_match_2mass_wise, num_match_2mass_galex, num_match_wise_galex)

Coordinate Match (2MASS-WISE, 2MASS-GALEX, WISE-GALEX) 41 15 18


In [11]:
data_2mass

ra,dec,clon,clat,err_maj,err_min,err_ang,designation,j_m,j_cmsig,j_msigcom,j_snr,h_m,h_cmsig,h_msigcom,h_snr,k_m,k_cmsig,k_msigcom,k_snr,ph_qual,rd_flg,bl_flg,cc_flg,ndet,gal_contam,mp_flg,hemis,xdate,scan,glon,glat,a,dist_opt,phi_opt,b_m_opt,vr_m_opt,nopt_mchs,ext_key,dist,angle,j_h,h_k,j_k,id
deg,deg,,,arcs,arcs,deg,,mag,mag,mag,,mag,mag,mag,,mag,mag,mag,,,,,,,,,,,,deg,deg,,arcs,deg,mag,mag,,,arcs,deg,,,,
float64,float64,object,object,float64,float64,int32,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,object,object,object,object,int32,int32,object,object,int32,float64,float64,object,float64,int32,float64,float64,int32,int32,float64,float64,float64,float64,float64,object
343.491,16.148,22h53m57.75s,16d08m53.63s,0.07,0.06,90,22535774+1608536,14.494,0.027,0.03,61.6,13.855,0.029,0.03,52.7,13.061,0.026,0.027,55.9,AAA,222,111,000,665566,0,0,n,1998-10-01,73,86.111,-38.184,U,0.0,115,14.8,14.1,1,--,0.072344,5.486687,0.639,0.794,1.433,0
343.492,16.152,22h53m58.16s,16d09m06.78s,0.07,0.06,90,22535816+1609067,11.844,0.017,0.021,707.7,11.339,0.021,0.023,535.0,11.239,0.019,0.021,299.1,AAA,222,111,000,666666,0,0,n,1998-10-01,73,86.115,-38.182,U,0.8,104,14.3,13.0,1,--,14.509797,24.348431,0.505,0.1,0.605,1
343.490,16.152,22h53m57.56s,16d09m08.57s,0.27,0.25,106,22535755+1609085,16.164,0.094,0.095,13.2,16.003,0.177,0.177,7.3,15.724,0.237,0.238,4.8,ACD,222,111,ccc,060606,0,0,n,1998-10-01,73,86.113,-38.18,U,1.2,284,16.7,16.1,1,--,15.26853,349.482477,0.161,0.279,0.44,2
343.493,16.141,22h53m58.37s,16d08m27.05s,0.07,0.06,90,22535836+1608270,14.8,0.035,0.037,46.5,14.103,0.038,0.039,42.0,13.838,0.04,0.041,27.3,AAA,222,111,000,666646,0,0,n,1998-10-01,73,86.108,-38.191,U,1.0,172,18.7,16.9,1,--,27.958175,161.480286,0.697,0.265,0.962,3
343.497,16.142,22h53m59.33s,16d08m32.80s,0.08,0.06,90,22535933+1608328,15.616,0.066,0.067,21.9,15.064,0.078,0.078,17.3,14.725,0.092,0.092,12.1,AAA,222,111,000,562616,0,0,n,1998-10-01,73,86.114,-38.192,U,0.5,93,19.7,17.8,1,--,30.832877,132.306841,0.552,0.339,0.891,4
343.499,16.143,22h53m59.74s,16d08m33.12s,0.28,0.26,133,22535974+1608331,16.162,0.09,0.091,13.3,15.66,0.12,0.12,10.0,15.675,0.226,0.227,5.0,ABD,222,111,000,061606,0,0,n,1998-10-01,73,86.116,-38.193,U,0.5,142,17.4,16.7,1,--,35.271661,125.401616,0.502,-0.015,0.487,5
343.501,16.158,22h54m00.21s,16d09m29.41s,0.22,0.21,86,22540021+1609294,16.827,0.154,0.155,7.2,16.046,0.163,0.164,7.0,15.275,--,--,--,BCU,220,110,000,060600,0,0,n,1998-10-01,73,86.129,-38.181,0,--,--,--,--,0,--,50.426614,44.683728,0.781,--,--,6
343.500,16.165,22h53m59.94s,16d09m54.03s,0.07,0.06,90,22535994+1609540,14.215,0.025,0.028,79.7,13.795,0.026,0.027,55.7,13.641,0.046,0.047,32.7,AAA,222,111,000,666666,0,0,n,1998-10-01,73,86.132,-38.175,U,0.7,77,16.4,15.4,1,--,68.22223,27.579842,0.42,0.154,0.574,7
